In [ ]:
args = "{ 'xsec_inj': '1e-22', 'trials': '1', 'mass': '100' }"

import ast
args = ast.literal_eval(args)

In [ ]:
cur_dir = '/tigress/somalwar/SubhalosFresh/'

In [ ]:
# Tigress dirs
import os, sys
import copy

import argparse
import numpy as np
import json
from scipy.optimize import minimize

# NPTFit modules
from NPTFit import nptfit # module for performing scan
from NPTFit import create_mask as cm # module for creating the mask
import pandas as pd
import healpy as hp

sys.path.append(cur_dir)
sys.path.append(cur_dir+"/Code/")
# My Modules
from Recxsec_modules_NP import makeMockData
from AssortedFunctions import getEnergyBinnedMaps

In [ ]:
nside = 128
npix = hp.nside2npix(nside)
ebins = 2*np.logspace(-1,3,41)[0:41]
my_iebins = [10, 15, 20, 40]

In [ ]:
xi = float(args['xsec_inj'])
mass = float(args['mass'])
trials = int(args['trials'])

In [ ]:
pscmask = np.array(np.load(cur_dir+'/fermi_data/fermidata_pscmask.npy'), dtype=bool)
mask = cm.make_mask_total(band_mask = True, band_mask_range = 5, mask_ring = True, inner = 20, outer = 180, custom_mask = pscmask)

In [ ]:
exposure_ebins = getEnergyBinnedMaps('/maps/exposure', cur_dir, my_iebins, ave=True, int32=False, nside=nside)
dif_ebins = getEnergyBinnedMaps('/maps/dif', cur_dir, my_iebins, ave=False, int32=False, nside=nside)
iso_ebins = getEnergyBinnedMaps('/maps/iso', cur_dir, my_iebins, ave=False, int32=False, nside=nside)
psc_ebins = getEnergyBinnedMaps('/maps/psc', cur_dir, my_iebins, ave=False, int32=False, nside=nside)
fermi_data_ebins = getEnergyBinnedMaps('/maps/data', cur_dir, my_iebins, ave=False, int32=True, nside=nside)

In [ ]:
bkg_fit = []
for ib, b in enumerate(my_iebins[:-1]):
    n_bkg = nptfit.NPTF(tag='norm')
    n_bkg.load_data(fermi_data_ebins[ib].copy().astype(np.int32), exposure_ebins[ib].copy())
    n_bkg.load_mask(mask)
    
    n_bkg.add_template(dif_ebins[ib].copy(), 'dif')
    n_bkg.add_template(iso_ebins[ib].copy(), 'iso')
    n_bkg.add_template(psc_ebins[ib].copy(), 'psc')
    
    n_bkg.add_poiss_model('dif', '$A_\mathrm{dif}$', [0,20], False)
    n_bkg.add_poiss_model('iso', '$A_\mathrm{iso}$', [0,20], False)
    n_bkg.add_poiss_model('psc', '$A_\mathrm{psc}$', [0,20], False)
    
    n_bkg.configure_for_scan()
    
    bkg_min = minimize( lambda args: -n_bkg.ll([*args]), 
                       [ 0.89, 5, 0.03795109 ], method="SLSQP", bounds = [ [0,10], [0,10], [0,10] ], 
                       options={'ftol':1e-15, 'eps':1e-10, 'maxiter':5000, 'disp':True} )
    bkg_fit.append(bkg_min.x)

In [ ]:
x0 = 1e-22
for ib, b in enumerate(my_iebins[:-1]):
    for t in range(trials):
        blazars_t = np.load(cur_dir+"/BlazarMC/blazarMC_iebins-"+str(my_iebins[ib])+"to"+str(my_iebins[ib+1])+"_"+str(t)+".npy")*exposure_ebins[ib]
        
        subMC_t = np.zeros(npix)
        it = 0
        while(True):
            try:
                subMC_t += np.load(cur_dir+"/SubhaloMC/subhaloMC_ExactJ_Einasto_iebins-"+str(my_iebins[ib])+"to"+str(my_iebins[ib+1])+"_m-"+str(mass)+"GeV_x-1e-22_"+str(it)+"_"+str(t)+".npy")
                it+=1
            except:
                break
        subMC_t = np.mean(exposure_ebins[ib]) * (xi/x0) * subMC_t
        subMC_t[subMC_t > 1000] = 0
        data_t = makeMockData( subMC_t, blazars_t, bkg_fit[ib][0]*dif_ebins[ib], bkg_fit[ib][1]*iso_ebins[ib] )
        np.save(cur_dir+"/MC/MC_ExactJ_Einasto_iebins-"+str(my_iebins[ib])+"to"+str(my_iebins[ib+1])+"_m-"+str(mass)+"GeV_x-"+str(xi)+"_"+str(t), data_t)
        